# Focus Buddy Agent - Kaggle Capstone Submission

**Track:** Concierge Agents  
**Concepts:** Multi-agent systems, Tool calling, Session memory

This notebook demonstrates the Focus Buddy concierge agent using direct Anthropic API calls.

In [ ]:
# Install dependencies (Kaggle environment)
!pip install anthropic pydantic -q

In [ ]:
import os
from anthropic import Anthropic
from typing import List, Optional
from pydantic import BaseModel
import re

# Set your API key using Kaggle Secrets
# Go to Kaggle: Add-ons → Secrets → Add a new secret named 'ANTHROPIC_API_KEY'
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("ANTHROPIC_API_KEY")

client = Anthropic(api_key=api_key)

## Tool Definitions

In [ ]:
# Data models
class Task(BaseModel):
    """Structured task representation"""
    title: str
    estimated_minutes: int = 10
    deadline: Optional[str] = None
    priority_score: float = 0.0
    completed: bool = False


class ScheduledBlock(BaseModel):
    """A time block in the focus schedule"""
    start_minute: int
    end_minute: int
    task_title: str


def parse_tasks(raw_text: str) -> List[Task]:
    """Parse messy task input into structured Task objects."""
    tasks = []
    lines = re.split(r'[\n\r]+|(?:^|\s)[-•*]\s+|\d+\.\s+', raw_text)
    
    for line in lines:
        line = line.strip()
        if not line or len(line) < 3:
            continue
            
        time_match = re.search(r'\((\d+)\s*min\)|\b(\d+)m\b', line, re.IGNORECASE)
        estimated_minutes = 10
        
        if time_match:
            estimated_minutes = int(time_match.group(1) or time_match.group(2))
            line = re.sub(r'\(?\d+\s*min?\)?', '', line, flags=re.IGNORECASE).strip()
        
        deadline = None
        deadline_match = re.search(r'\b(due|deadline|by)\s+([a-zA-Z]+\s+\d+|\d+/\d+|today|tomorrow)', line, re.IGNORECASE)
        if deadline_match:
            deadline = deadline_match.group(2)
        
        tasks.append(Task(
            title=line,
            estimated_minutes=estimated_minutes,
            deadline=deadline
        ))
    
    return tasks


def prioritize_tasks(tasks: List[Task]) -> List[Task]:
    """Sort tasks by priority score."""
    for task in tasks:
        score = 0.0
        
        if task.deadline:
            deadline_lower = task.deadline.lower()
            if 'today' in deadline_lower or 'urgent' in deadline_lower:
                score += 3
            elif 'tomorrow' in deadline_lower:
                score += 2
        
        if task.estimated_minutes <= 10:
            score += 2
        
        title_lower = task.title.lower()
        if any(word in title_lower for word in ['urgent', 'important', 'blocking', 'asap', 'critical']):
            score += 2
        if any(word in title_lower for word in ['bug', 'fix', 'broken', 'error']):
            score += 1.5
        
        score += (60 - min(task.estimated_minutes, 60)) / 100
        task.priority_score = score
    
    return sorted(tasks, key=lambda t: t.priority_score, reverse=True)


def create_focus_schedule(tasks: List[Task], available_minutes: int = 25) -> List[ScheduledBlock]:
    """Build a realistic schedule that fits in available_minutes."""
    schedule = []
    current_minute = 0
    buffer_minutes = 3
    usable_minutes = available_minutes - buffer_minutes
    
    for task in tasks:
        if current_minute + task.estimated_minutes > usable_minutes:
            remaining = usable_minutes - current_minute
            if remaining >= 5:
                schedule.append(ScheduledBlock(
                    start_minute=current_minute,
                    end_minute=current_minute + remaining,
                    task_title=f"{task.title} (partial - {remaining} min)"
                ))
                current_minute += remaining
            break
        
        schedule.append(ScheduledBlock(
            start_minute=current_minute,
            end_minute=current_minute + task.estimated_minutes,
            task_title=task.title
        ))
        current_minute += task.estimated_minutes
        
        if len(schedule) >= 4:
            break
    
    if current_minute < available_minutes:
        schedule.append(ScheduledBlock(
            start_minute=current_minute,
            end_minute=available_minutes,
            task_title="Wrap up & notes for next session"
        ))
    
    return schedule


def execute_tool(tool_name: str, tool_input: dict):
    """Execute tool and return results."""
    if tool_name == "parse_tasks":
        result = parse_tasks(tool_input["raw_text"])
        return [task.dict() for task in result]
    
    elif tool_name == "prioritize_tasks":
        tasks = [Task(**t) for t in tool_input["tasks"]]
        result = prioritize_tasks(tasks)
        return [task.dict() for task in result]
    
    elif tool_name == "create_focus_schedule":
        tasks = [Task(**t) for t in tool_input["tasks"]]
        available = tool_input.get("available_minutes", 25)
        result = create_focus_schedule(tasks, available)
        return [block.dict() for block in result]
    
    return {"error": f"Unknown tool: {tool_name}"}

print("✓ Tools defined successfully")

## Agent System Prompt

In [ ]:
FOCUS_BUDDY_PROMPT = """
You are *Focus Buddy*, a concierge AI agent that helps a user turn a messy to-do list into a realistic 25–30 minute deep-work sprint plan.

You are part of a **multi-agent system** with two internal roles:
1. **Planner Agent** – parses tasks, prioritizes, and creates schedules
2. **Coach Agent** – presents plans, motivates, and checks in

## Workflow
1. Ask for tasks and time window (default 25 min) and energy level
2. Use parse_tasks → prioritize_tasks → create_focus_schedule
3. Present plan with Summary, Timeline, Checklist, Check-in instructions
4. After session, track completion and suggest improvements

## Tools Available
- parse_tasks(raw_text) → structured tasks
- prioritize_tasks(tasks) → sorted by urgency/importance
- create_focus_schedule(tasks, available_minutes) → time blocks

## Style
- Concise, practical, no fluff
- Use bullet lists and timelines
- Limit to 3-5 tasks per 25 minutes
- If user seems overwhelmed, reduce scope
"""

# Tool schemas for Claude
tools = [
    {
        "name": "parse_tasks",
        "description": "Parse messy task text into structured Task objects",
        "input_schema": {
            "type": "object",
            "properties": {
                "raw_text": {"type": "string", "description": "Raw task input"}
            },
            "required": ["raw_text"]
        }
    },
    {
        "name": "prioritize_tasks",
        "description": "Sort tasks by priority",
        "input_schema": {
            "type": "object",
            "properties": {
                "tasks": {"type": "array", "items": {"type": "object"}}
            },
            "required": ["tasks"]
        }
    },
    {
        "name": "create_focus_schedule",
        "description": "Create time-bounded schedule",
        "input_schema": {
            "type": "object",
            "properties": {
                "tasks": {"type": "array", "items": {"type": "object"}},
                "available_minutes": {"type": "integer", "default": 25}
            },
            "required": ["tasks"]
        }
    }
]

print("✓ Agent configuration loaded")

## Demo Session

In [ ]:
def run_demo():
    """Run a demonstration of Focus Buddy."""
    messages = []
    
    # Simulated user input
    user_message = """
    I have 25 minutes and I'm pretty tired. Here's what I need to do:
    - Review 3 pull requests
    - Write API docs for /users endpoint (30 min)
    - Reply to Sarah about Q4 planning
    - Fix authentication bug (blocking QA)
    - Prep slides for Friday demo
    """
    
    print("USER INPUT:")
    print(user_message)
    print("\n" + "="*60 + "\n")
    
    messages.append({"role": "user", "content": user_message})
    
    # Call Claude
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        system=FOCUS_BUDDY_PROMPT,
        tools=tools,
        messages=messages
    )
    
    # Handle tool calls
    iteration = 0
    while response.stop_reason == "tool_use" and iteration < 10:
        iteration += 1
        tool_results = []
        
        for block in response.content:
            if block.type == "tool_use":
                print(f"🔧 Tool called: {block.name}")
                result = execute_tool(block.name, block.input)
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": block.id,
                    "content": str(result)
                })
        
        messages.append({"role": "assistant", "content": response.content})
        messages.append({"role": "user", "content": tool_results})
        
        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            system=FOCUS_BUDDY_PROMPT,
            tools=tools,
            messages=messages
        )
    
    # Display final response
    print("\nFOCUS BUDDY RESPONSE:")
    print("="*60)
    for block in response.content:
        if hasattr(block, "text"):
            print(block.text)

# Run the demo
run_demo()

## Concepts Demonstrated

This notebook demonstrates three key concepts from the AI Agents Intensive:

1. **Multi-agent systems**: Planner and Coach agents coordinate through shared context
2. **Tool use**: Three Python functions (parse, prioritize, schedule) are called by the agent
3. **Session memory**: The agent tracks the current focus plan and completion status

## Value Proposition

Focus Buddy reduces decision fatigue for knowledge workers by:
- Converting messy lists into actionable 25-minute plans
- Prioritizing realistically based on time and energy
- Creating momentum through small, completable goals
- Enabling repeatable focus cycles throughout the day